In [1]:
import pandas as pd 
import numpy as np
import datetime

In [2]:
df_2019 = pd.read_csv('C:\\Users\\yanis\\Downloads\\Foot test\\2019data.csv')

In [3]:
print(df_2019)

    Div        Date   Time     HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  \
0    F1  09/08/2019  19:45       Monaco         Lyon     0     3   A     0   
1    F1  10/08/2019  16:30    Marseille        Reims     0     2   A     0   
2    F1  10/08/2019  19:00       Angers     Bordeaux     3     1   H     3   
3    F1  10/08/2019  19:00        Brest     Toulouse     1     1   D     1   
4    F1  10/08/2019  19:00        Dijon   St Etienne     1     2   A     1   
5    F1  10/08/2019  19:00  Montpellier       Rennes     0     1   A     0   
6    F1  10/08/2019  19:00         Nice       Amiens     2     1   H     1   
7    F1  11/08/2019  14:00        Lille       Nantes     2     1   H     1   
8    F1  11/08/2019  16:00   Strasbourg         Metz     1     1   D     1   
9    F1  11/08/2019  20:00     Paris SG        Nimes     3     0   H     1   
10   F1  16/08/2019  19:45         Lyon       Angers     6     0   H     3   
11   F1  17/08/2019  16:30       Nantes    Marseille     0     0

In [4]:
def get_winning_team(row):
    team_1 = row['HomeTeam']
    team_2 = row['AwayTeam']
    if row['FTR'] == "H":
        return team_1
    elif row['FTR'] == "D" :
        return "Draw"
    else :
        return team_2

In [5]:
def get_losing_team(row):
    team_1 = row['HomeTeam']
    team_2 = row['AwayTeam']
    if row['FTR'] == "A":
        return team_1
    elif row['FTR'] == "D" :
        return "Draw"
    else :
        return team_2
df_2019['WinningTeam'] = df_2019.apply(get_winning_team, axis=1)
df_2019['LosingTeam'] = df_2019.apply(get_losing_team, axis=1)

In [6]:
F1Teams=['Lyon','Nantes','Reims','Bordeaux','Paris SG','Lille','Marseille','Nimes','St Etienne','Montpellier','Monaco','Angers','Nice','Toulouse','Amiens','Rennes','Brest','Dijon','Strasbourg','Metz']

In [7]:
df19 = pd.DataFrame(F1Teams,columns =['Teams']) 

In [8]:
# Played Games
for i in df19.index:
    team = df19.loc[i,"Teams"]
    home_game = df_2019.groupby("HomeTeam").count().loc[team,"Div"]
    away_game = df_2019.groupby("AwayTeam").count().loc[team,"Div"]
    df19.loc[i,"Games"] = home_game + away_game

In [9]:
# Number of wins
for i in df19.index:
    team = df19.loc[i,"Teams"]
    wins = df_2019.groupby("WinningTeam").count().loc[team,"Div"]
    df19.loc[i,"Wins"] =wins
df19['WinningRatio']= df19['Wins']/df19['Games']

In [10]:
# Number of losses 
for i in df19.index:
    team = df19.loc[i,"Teams"]
    losses = df_2019.groupby("LosingTeam").count().loc[team,"Div"]
    df19.loc[i,"Losses"] = losses
df19['LosingRatio']= df19['Losses']/df19['Games']

In [11]:
# Number of Draws
df19['Draws'] = abs((df19['Wins'] + df19['Losses']) - df19['Games'])
df19['DrawRatio'] = df19['Draws']/df19['Games']

In [12]:
df19 = df19[['Teams','Games','Wins','Losses','Draws','WinningRatio','LosingRatio','DrawRatio']]

In [13]:
df_2019["home_win"] = df_2019["FTR"] == "H"

In [14]:
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

In [58]:
for n in df19.index :
    team = df19.loc[n,"Teams"]
    lossesbefore = "Games lost before by {}".format(team)
    winsbefore = "Games won before by {}".format(team)
x = df_2019[["B365H","B365A","HST","AST",lossesbefore,winsbefore]]
y = df_2019["home_win"]
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [59]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

C:\Users\yanis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [60]:
print(lr.coef_)
print(lr.intercept_)

[[-0.46954251  0.02434305  0.37914607 -0.31462259  0.00137751  0.20721005]]
[0.06557296]


In [61]:
y_pred = lr.predict(x_test)

In [62]:
confusion_matrix(y_test, y_pred)

array([[15, 10],
       [ 6, 16]], dtype=int64)

In [63]:
# Use score method to get accuracy of model
score = lr.score(x_test, y_test)
print(score)

0.6595744680851063


In [64]:
for i in df_2019.index:
    if df_2019.loc[i,"home_win"] == True :
        df_2019.loc[i,"home_win"] = 1
    else :
        df_2019.loc[i,"home_win"] = 0

In [65]:
df_2019["away_win"] = df_2019["FTR"] == "A"

In [66]:
for i in df_2019.index:
    if df_2019.loc[i,"away_win"] == True :
        df_2019.loc[i,"away_win"] = 1
    else :
        df_2019.loc[i,"away_win"] = 0

In [67]:
for i  in df_2019.index :
    datetimecorr1 = datetime.datetime.strptime(df_2019.loc[i,'Date'], '%d/%m/%Y')
    df_2019.loc[i,"Datetime"] = datetimecorr1.date()

In [68]:
for i in df_2019.index:
    date = df_2019.loc[i,"Datetime"]
    temp_df = df_2019[df_2019["Datetime"] < date]
    df_2019.loc[i,"game_won_before_h"] = temp_df["home_win"].sum()

In [69]:
for i in df_2019.index :
    date = df_2019.loc[i,"Datetime"]
    temp_df = df_2019[df_2019["Datetime"] < date]
    df_2019.loc[i,"game_won_before_a"] = temp_df["away_win"].sum()

In [70]:
totalhomegoals = df_2019["FTHG"].sum()
totalawaygoals = df_2019["FTAG"].sum()
totalgoals = totalhomegoals + totalawaygoals
totalplayedgames = df19["Games"].sum() /2
avghomegoals = totalhomegoals / totalplayedgames
avgawaygoals = totalawaygoals / totalplayedgames
avggoals = totalgoals / totalplayedgames
print(avghomegoals,avgawaygoals,avggoals)

1.5401069518716577 0.9572192513368984 2.4973262032085564


In [71]:
datelist = list(df_2019["Date"])

In [72]:
print(datelist)

['09/08/2019', '10/08/2019', '10/08/2019', '10/08/2019', '10/08/2019', '10/08/2019', '10/08/2019', '11/08/2019', '11/08/2019', '11/08/2019', '16/08/2019', '17/08/2019', '17/08/2019', '17/08/2019', '17/08/2019', '17/08/2019', '17/08/2019', '18/08/2019', '18/08/2019', '18/08/2019', '24/08/2019', '24/08/2019', '24/08/2019', '24/08/2019', '25/08/2019', '25/08/2019', '25/08/2019', '27/08/2019', '28/08/2019', '28/08/2019', '30/08/2019', '31/08/2019', '31/08/2019', '31/08/2019', '31/08/2019', '31/08/2019', '01/09/2019', '01/09/2019', '01/09/2019', '01/09/2019', '13/09/2019', '13/09/2019', '14/09/2019', '14/09/2019', '14/09/2019', '14/09/2019', '14/09/2019', '15/09/2019', '15/09/2019', '15/09/2019', '20/09/2019', '21/09/2019', '21/09/2019', '21/09/2019', '21/09/2019', '21/09/2019', '21/09/2019', '22/09/2019', '22/09/2019', '22/09/2019', '24/09/2019', '24/09/2019', '25/09/2019', '25/09/2019', '25/09/2019', '25/09/2019', '25/09/2019', '25/09/2019', '25/09/2019', '25/09/2019', '28/09/2019', '28/0

In [73]:
# Number of Home wins
for i in df19.index:
    team = df19.loc[i,"Teams"]
    winningratioh = df_2019.groupby("HomeTeam").mean().loc[team,"home_win"]
    df19.loc[i,"Homewins"] = winningratioh

In [74]:
# Number of Away wins
for i in df19.index:
    team = df19.loc[i,"Teams"]
    winningratioa = df_2019.groupby("AwayTeam").mean().loc[team,"away_win"]
    df19.loc[i,"Awaywins"] = winningratioa

In [75]:
# Number of Home Goals
for i in df19.index:
    team = df19.loc[i,"Teams"]
    homegoalsh = df_2019.groupby("HomeTeam").mean().loc[team,"FTHG"]
    df19.loc[i,"N° home goals per game"] = homegoalsh

In [76]:
# Number of Away Goals
for i in df19.index:
    team = df19.loc[i,"Teams"]
    awaygoalsa = df_2019.groupby("AwayTeam").mean().loc[team,"FTAG"]
    df19.loc[i,"N° away goals per game"] = awaygoalsa

In [77]:
# Number of Home Goals conceded
for i in df19.index:
    team = df19.loc[i,"Teams"]
    homegoalsa = df_2019.groupby("HomeTeam").mean().loc[team,"FTAG"]
    df19.loc[i,"N° home goals conceded per game"] = homegoalsa

In [78]:
# Number of Away Goals conceded
for i in df19.index:
    team = df19.loc[i,"Teams"]
    awaygoalsh = df_2019.groupby("AwayTeam").mean().loc[team,"FTHG"]
    df19.loc[i,"N° away goals conceded per game"] = awaygoalsh

In [79]:
# Number of Goals per game
for i in df19.index:
    team = df19.loc[i,"Teams"]
    homegoalsa = df_2019.groupby("HomeTeam").mean().loc[team,"FTHG"]
    awaygoalsa = df_2019.groupby("AwayTeam").mean().loc[team,"FTAG"]
    homegoalsa = df_2019.groupby("HomeTeam").mean().loc[team,"FTAG"]
    awaygoalsh = df_2019.groupby("AwayTeam").mean().loc[team,"FTHG"]
    df19.loc[i,"N° goals per game"] = (homegoalsa + awaygoalsa + homegoalsh + awaygoalsh)/2

In [80]:
df_2019["AvgH"].sum() / totalplayedgames

2.542139037433155

In [81]:
df_2019["AvgA"].sum() / totalplayedgames

4.40855614973262

In [82]:
df_2019["AvgD"].sum() / totalplayedgames

3.7583422459893043

In [83]:
df_2019["Avg>2.5"].sum() / totalplayedgames

2.08096256684492

In [84]:
df_2019["Avg<2.5"].sum() / totalplayedgames

1.8575401069518718

In [85]:
df19

,Teams,Games,Wins,Losses,Draws,WinningRatio,LosingRatio,DrawRatio,Homewins,Awaywins,N° home goals per game,N° away goals per game,N° home goals conceded per game,N° away goals conceded per game,N° goals per game,Teamsc
0,Lyon,19.0,7.0,7.0,5.0,0.368421,0.368421,0.263158,0.333333,0.400000,1.222222,1.800000,0.666667,1.200000,2.444444,Lyon
1,Nantes,19.0,9.0,8.0,2.0,0.473684,0.421053,0.105263,0.600000,0.333333,1.000000,0.777778,0.700000,1.222222,1.961111,Nantes
2,Reims,18.0,7.0,4.0,7.0,0.388889,0.222222,0.388889,0.300000,0.500000,0.900000,0.875000,0.500000,0.625000,1.611111,Reims
3,Bordeaux,19.0,7.0,7.0,5.0,0.368421,0.368421,0.263158,0.444444,0.300000,1.666667,1.300000,0.777778,1.500000,2.400000,Bordeaux
4,Paris SG,18.0,15.0,3.0,0.0,0.833333,0.166667,0.000000,0.888889,0.777778,2.666667,2.111111,0.333333,0.777778,2.222222,Paris_SG
5,Lille,19.0,9.0,6.0,4.0,0.473684,0.315789,0.210526,0.800000,0.111111,1.800000,0.666667,0.500000,1.777778,2.083333,Lille
6,Marseille,19.0,11.0,3.0,5.0,0.578947,0.157895,0.263158,0.700000,0.444444,1.700000,1.333333,0.900000,1.333333,2.394444,Marseille
7,Nimes,18.0,2.0,10.0,6.0,0.111111,0.555556,0.333333,0.250000,0.000000,0.875000,0.600000,1.250000,2.200000,2.636111,Nimes
8,St Etienne,19.0,7.0,8.0,4.0,0.368421,0.421053,0.210526,0.333333,0.400000,1.222222,1.100000,1.222222,1.800000,2.672222,St_Etienne
9,Montpellier,19.0,7.0,6.0,6.0,0.368421,0.315789,0.315789,0.700000,0.000000,1.900000,0.777778,0.800000,1.222222,2.011111,Montpellier


In [86]:
# Wins of each team 1-0
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        name1 = "Wins of {}".format(team)
        if df_2019.loc[i,"WinningTeam"] == team :
            df_2019.loc[i,name1] = 1
        else :
            df_2019.loc[i,name1] = 0

In [87]:
# Wins before game
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        name1 = "Wins of {}".format(team)
        name2 = "Games won before by {}".format(team)
        df_2019.loc[i,name2] = temp_df[name1].sum()

In [88]:
# Losses of each team 1-0
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        name1 = "Losses of {}".format(team)
        if df_2019.loc[i,"LosingTeam"] == team :
            df_2019.loc[i,name1] = 1
        else :
            df_2019.loc[i,name1] = 0

In [89]:
# Losses before game
for n in df19.index :
    for i in df_2019.index:
        team = df19.loc[n,"Teams"]
        date = df_2019.loc[i,"Datetime"]
        temp_df = df_2019[df_2019["Datetime"] < date]
        name1 = "Losses of {}".format(team)
        name2 = "Games lost before by {}".format(team)
        df_2019.loc[i,name2] = temp_df[name1].sum()

In [53]:
df19["Teamsc"] = df19["Teams"].apply(lambda x: x.replace(" ","_"))

In [54]:
df19

,Teams,Games,Wins,Losses,Draws,WinningRatio,LosingRatio,DrawRatio,Homewins,Awaywins,N° home goals per game,N° away goals per game,N° home goals conceded per game,N° away goals conceded per game,N° goals per game,Teamsc
0,Lyon,19.0,7.0,7.0,5.0,0.368421,0.368421,0.263158,0.333333,0.400000,1.222222,1.800000,0.666667,1.200000,2.444444,Lyon
1,Nantes,19.0,9.0,8.0,2.0,0.473684,0.421053,0.105263,0.600000,0.333333,1.000000,0.777778,0.700000,1.222222,1.961111,Nantes
2,Reims,18.0,7.0,4.0,7.0,0.388889,0.222222,0.388889,0.300000,0.500000,0.900000,0.875000,0.500000,0.625000,1.611111,Reims
3,Bordeaux,19.0,7.0,7.0,5.0,0.368421,0.368421,0.263158,0.444444,0.300000,1.666667,1.300000,0.777778,1.500000,2.400000,Bordeaux
4,Paris SG,18.0,15.0,3.0,0.0,0.833333,0.166667,0.000000,0.888889,0.777778,2.666667,2.111111,0.333333,0.777778,2.222222,Paris_SG
5,Lille,19.0,9.0,6.0,4.0,0.473684,0.315789,0.210526,0.800000,0.111111,1.800000,0.666667,0.500000,1.777778,2.083333,Lille
6,Marseille,19.0,11.0,3.0,5.0,0.578947,0.157895,0.263158,0.700000,0.444444,1.700000,1.333333,0.900000,1.333333,2.394444,Marseille
7,Nimes,18.0,2.0,10.0,6.0,0.111111,0.555556,0.333333,0.250000,0.000000,0.875000,0.600000,1.250000,2.200000,2.636111,Nimes
8,St Etienne,19.0,7.0,8.0,4.0,0.368421,0.421053,0.210526,0.333333,0.400000,1.222222,1.100000,1.222222,1.800000,2.672222,St_Etienne
9,Montpellier,19.0,7.0,6.0,6.0,0.368421,0.315789,0.315789,0.700000,0.000000,1.900000,0.777778,0.800000,1.222222,2.011111,Montpellier


In [55]:
for n in df19.index :
    team = df19.loc[n,"Teamsc"]
    nameteam = "df19_{}".format(team)
    for i in df_2019.index:
        df19_teamh = pd.DataFrame(df_2019.loc[df_2019.HomeTeam == team])
    for i in df_2019.index:
        df19_teama = pd.DataFrame(df_2019.loc[df_2019.AwayTeam == team])
    framesteam = [df19_teama, df19_teamh]
    result = pd.concat(framesteam)
    exec("%s = result.sort_index()" % (nameteam))

In [56]:
for n in df19.index :
    team = df19.loc[n,"Teamsc"]
    nameteam = "df19_{}".format(team)
    exec("%s['WINS']= %s['WinningTeam'] == team" %(nameteam,nameteam))

In [57]:
df19_Angers

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,Games lost before by Angers,Games lost before by Nice,Games lost before by Toulouse,Games lost before by Amiens,Games lost before by Rennes,Games lost before by Brest,Games lost before by Dijon,Games lost before by Strasbourg,Games lost before by Metz,WINS
2,F1,10/08/2019,19:00,Angers,Bordeaux,3,1,H,3,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
10,F1,16/08/2019,19:45,Lyon,Angers,6,0,H,3,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,False
21,F1,24/08/2019,19:00,Angers,Metz,3,0,H,2,0,...,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,True
32,F1,31/08/2019,19:00,Angers,Dijon,2,0,H,0,0,...,1.0,1.0,1.0,2.0,0.0,0.0,3.0,1.0,2.0,True
40,F1,13/09/2019,18:00,Lille,Angers,2,1,H,1,0,...,1.0,1.0,1.0,3.0,1.0,1.0,4.0,1.0,2.0,False
58,F1,22/09/2019,16:00,Angers,St Etienne,4,1,H,0,1,...,2.0,2.0,2.0,3.0,1.0,1.0,5.0,2.0,4.0,True
68,F1,25/09/2019,18:00,Toulouse,Angers,0,2,A,0,0,...,2.0,3.0,2.0,3.0,1.0,1.0,5.0,2.0,4.0,True
72,F1,28/09/2019,19:00,Angers,Amiens,1,1,D,0,1,...,2.0,3.0,3.0,4.0,2.0,1.0,5.0,3.0,4.0,False
81,F1,05/10/2019,16:30,Paris SG,Angers,4,0,H,2,0,...,2.0,3.0,3.0,4.0,2.0,2.0,5.0,3.0,4.0,False
92,F1,19/10/2019,19:00,Angers,Brest,0,1,A,0,0,...,3.0,5.0,4.0,4.0,3.0,2.0,5.0,4.0,5.0,False


In [94]:
df19_Angers.to_csv('C:\\Users\\yanis\\Downloads\\Foot test\\Clubs_2019\\Angers_2019')

In [105]:
for n in df19.index :
    team = df19.loc[n,"Teamsc"]
    nameteam = "df19_{}".format(team)
    exec("%s.to_csv(r'C:\\Users\\yanis\\Downloads\\Foot test\\Clubs_2019\\%s_2019')" %(nameteam,nameteam))